# LLM Agents
This is the third notebook for the LLM section of Comp 255.

1. [Is RAG an agent?](#is-rag-an-agent)
2. [Tool use with AgentBot](#tool-use-with-agentbot)
   - [Exercise: Make a RAG tool](#exercise-make-a-rag-tool)
3. Fine-tuning: Access [this notebook](https://colab.research.google.com/drive/1HB5c6RkyOljXXghih11Y8Xbe51uhO2s2?usp=sharing) for this section

In [2]:
from llamabot import QueryBot, AgentBot, tool
from pathlib import Path

sft_model = "qwen2.5:1.5b"
agent_model = "qwen2.5:7b"

/opt/homebrew/Caskroom/miniconda/base/envs/llamabot/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Is RAG an agent?
Let's set up our RAG workflow from last time and consider whether this meets the definition of "agent"

In [ ]:
# setting the data back up
doc1 = "Languages with statutory recognition in Papua New Guinea \
are Tok Pisin, English, Hiri Motu, and \
Papua New Guinean Sign Language"

doc2 = "The only language with statutory recognition in France \
is French"

# write these to a temporary file
with open("doc1.txt", "w") as f:
    f.write(doc1)

with open("doc2.txt", "w") as f:
    f.write(doc2)

In [ ]:
system_message = "You are a helpful assistant that answers questions \
    based on the provided documents."
doc_paths = [Path("doc1.txt"), Path("doc2.txt")]

query_completer = QueryBot(
    system_prompt=system_message,
    model_name = f"ollama_chat/{sft_model}",
    collection_name="documents",
    document_paths=doc_paths
)

In [ ]:
response = query_completer('Hi how are you today!')


Is this an agent? It did respond reasonably, but it seems to be pretty preoccupied with the RAG use-case.  An agent should have "agency"; it should be able to decide the right action to take.  Sometimes that might involve the RAG workflow, but other times it won't.

## Tool use with AgentBot

Just like you would use Google to figure out what languages are spoken in a particular country, the agent can be designed to use "tools" to answer a question.  In that case, the retrieval workflow above could just be considered a tool.

A tool, in this case, is essentially just a function or set of functions.  Since the LLM can't directly run a function, it has to generate a response that can then be parsed into what function to call and what parameters to use.

Here we create a simple tool for calculating the total of a restaurant bill with tip.  Then we create an `AgentBot` instance, which uses a specially-designed prompt with information about the available tools.  With this information, the LLM can decide whether or not to use a tool and, if it uses a tool, what parameters to supply.

NOTE: This is a COMPLEX task and requires a bigger model than we've been using thus far.  Complex workflows are likely outside the capabilities of even this larger model.

In [3]:
# adapted from https://github.com/ericmjl/llamabot/blob/main/docs/tutorials/agentbot.md
# llamabot.tool decorator
@tool
def calculate_total_with_tip(bill_amount: float, tip_rate: float) -> float:
    if tip_rate < 0 or tip_rate > 1.0:
        raise ValueError("Tip rate must be between 0 and 1.0")
    return bill_amount * (1 + tip_rate)

# Create the bot
bot = AgentBot(
    system_prompt="You are my assistant with respect to restaurant bills.",
    functions=[calculate_total_with_tip, ],
    model_name=f"ollama_chat/{agent_model}",
    #model_name='gpt-4o'  
)

In [4]:
# look at what is being provided to the model
print(calculate_total_with_tip.json_schema)

{'properties': {'function_name': {'default': 'calculate_total_with_tip', 'description': 'Name of the function', 'title': 'Function Name', 'type': 'string'}, 'source_code': {'default': '@tool\ndef calculate_total_with_tip(bill_amount: float, tip_rate: float) -> float:\n    if tip_rate < 0 or tip_rate > 1.0:\n        raise ValueError("Tip rate must be between 0 and 1.0")\n    return bill_amount * (1 + tip_rate)\n', 'description': 'Source code of the function', 'title': 'Source Code', 'type': 'string'}, 'bill_amount': {'title': 'Bill Amount', 'type': 'number'}, 'tip_rate': {'title': 'Tip Rate', 'type': 'number'}}, 'required': ['bill_amount', 'tip_rate'], 'title': 'Calculate_Total_With_TipParameters', 'type': 'object'}


In [5]:
bot.tools

{'agent_finish': <function llamabot.bot.agentbot.agent_finish(message: Any) -> str>,
 'return_error': <function llamabot.bot.agentbot.return_error(message: Any) -> str>,
 'calculate_total_with_tip': <function __main__.calculate_total_with_tip(bill_amount: float, tip_rate: float) -> float>}

In [6]:
# system prompt has been extended with additional information
bot.decision_bot.system_prompt.content

"You are my assistant with respect to restaurant bills. Given the following message and available tools, pick the next tool that you need to call and the arguments that you need for it. You have access to previously cached results - use them when appropriate by specifying their hash key in use_cached_results. Any arguments that you do not know ahead of time, just leave blank. Only call tools that are relevant to the user's message. If the task is complete, use the agent_finish tool."

In [ ]:
calculate_total_only_prompt = "My dinner was $2300 without tips. Calculate my total with an 18% tip."
response = bot(calculate_total_only_prompt, max_iterations=4)
print(response.content)

This will *usually* work.  The issue with small models (7 billion parameters in this case) is that they can sometimes output strange things, which will cause errors.  Sometimes the models can self-correct, but sometimes not.

The bot now could be said to have "agency"; it can decide whether to use a tool or just go ahead and respond normally.

In [ ]:
response = bot('Hello how are you?')
print(response.content)

### Exercise: Make a RAG tool
Since you already have the syntax above, you can wrap our previous retrieval workflow in a tool function.  Try it out.